In [126]:
import pandas as pd
import yfinance as yf
import statsmodels.formula.api as smf
import pandas_datareader.data as web

In [127]:
RISKY_ASSET = 'AMZN'
START_DATE = '2010-12-31'
END_DATE = '2016-12-31'

In [134]:
# three factors 
df_three_factor = web.DataReader('F-F_Research_Data_Factors', 
                                 'famafrench', start=START_DATE)[0]
df_three_factor.index = df_three_factor.index.format()

# momentum factor
df_mom = web.DataReader('F-F_Momentum_Factor', 'famafrench', 
                        start=START_DATE)[0]
df_mom.index = df_mom.index.format()

# five factors
df_five_factor = web.DataReader('F-F_Research_Data_5_Factors_2x3', 
                                'famafrench', 
                                start=START_DATE)[0]
df_five_factor.index = df_five_factor.index.format()
df_five_factor
asset_df = yf.download(RISKY_ASSET, start=START_DATE, end=END_DATE, 
                       #auto_adjust=True
                       progress=False)
y = asset_df['Adj Close'].resample('M') \
                         .last() \
                         .pct_change() \
                         .dropna()

y.index = y.index.strftime('%Y-%m')
y.name = 'return'


# join all datasets on the index
five_factor_data = df_five_factor.join(y)

# rename columns
five_factor_data.columns = ['mkt', 'smb', 'hml', 'rmw', 'cma', 
                            'rf', 'rtn']

# divide everything (except returns) by 100
five_factor_data.loc[:, five_factor_data.columns != 'rtn'] /= 100

# select period of interest
five_factor_data = five_factor_data.loc[START_DATE:END_DATE]

# calculate excess returns
five_factor_data['excess_rtn'] = five_factor_data.rtn - five_factor_data.rf

five_factor_model = smf.ols(
    formula='excess_rtn ~ mkt + smb + hml + rmw + cma', 
    data=five_factor_data
).fit()

print(five_factor_model.summary())

C:\Users\clint\AppData\Local\Temp\ipykernel_19724\701520686.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_three_factor = web.DataReader('F-F_Research_Data_Factors',
C:\Users\clint\AppData\Local\Temp\ipykernel_19724\701520686.py:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_three_factor = web.DataReader('F-F_Research_Data_Factors',
C:\Users\clint\AppData\Local\Temp\ipykernel_19724\701520686.py:7: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df_mom = web.DataReader('F-F_Momentum_Factor', 'famafrench',
C:\Users\clint

                            OLS Regression Results                            
Dep. Variable:             excess_rtn   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.297
Method:                 Least Squares   F-statistic:                     7.008
Date:                Sat, 20 May 2023   Prob (F-statistic):           2.63e-05
Time:                        10:29:19   Log-Likelihood:                 96.476
No. Observations:                  72   AIC:                            -181.0
Df Residuals:                      66   BIC:                            -167.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0142      0.008      1.701      0.0

In [136]:
five_factor_data

,mkt,smb,hml,rmw,cma,rf,rtn,excess_rtn
2011-01,0.0199,-0.0238,0.0075,-0.0065,0.0077,0.0001,-0.057556,-0.057656
2011-02,0.0349,0.0165,0.0127,-0.0193,0.0087,0.0001,0.021516,0.021416
2011-03,0.0046,0.0263,-0.0185,0.0180,-0.0005,0.0001,0.039471,0.039371
2011-04,0.0290,-0.0050,-0.0249,0.0111,-0.0086,0.0000,0.087048,0.087048
2011-05,-0.0127,-0.0071,-0.0200,0.0191,-0.0150,0.0000,0.004494,0.004494
...,...,...,...,...,...,...,...,...
2016-08,0.0049,0.0170,0.0313,-0.0188,-0.0034,0.0002,0.013640,0.013440
2016-09,0.0025,0.0186,-0.0123,-0.0223,0.0003,0.0002,0.088603,0.088403
2016-10,-0.0202,-0.0404,0.0412,0.0096,0.0028,0.0002,-0.056717,-0.056917
2016-11,0.0486,0.0704,0.0819,-0.0021,0.0369,0.0001,-0.049695,-0.049795


In [137]:
rf = five_factor_data['rf'].mean()
market_premium = five_factor_data['Mkt-RF'].mean()
smb_premium = five_factor_data['SMB'].mean()
hml_premium = five_factor_data['HML'].mean()
#rmw_premium
#cma_premium       

expected_monthly_return = rf + b1 * market_premium + b2 * smb_premium + b3 * hml_premium 
expected_yearly_return = expected_monthly_return * 12
print("Expected yearly return: " + str(expected_yearly_return))

KeyError: 'Mkt-RF'